Drive Link for train sample data: https://drive.google.com/file/d/1R6HOBJc2368229_BsWBLV0-SbMQlpeIl/view?usp=sharing

test: https://drive.google.com/open?id=1r0qw7K4g2MFygXf8PYl2cuIltHqQJg9S

In [1]:
# from google.colab import drive
# drive.mount('/content/drive')

In [1]:
import os
import torch
from math import floor
# from torchvision import datasets, transforms
# from torch.utils.data import DataLoader
# import torchvision.models as models
from PIL import Image
from glob import glob
import numpy as np
# import matplotlib.pyplot as plt 
import pandas as pd
from torch.utils.data.sampler import SubsetRandomSampler
# import pickle as pkl
import torch.nn as nn
import torch.nn.functional as F
import torch.nn.init as init
from torch.nn.utils.rnn import pack_padded_sequence

# %matplotlib inline

In [2]:
use_cuda = torch.cuda.is_available()
if not use_cuda:
    print('CUDA is not available.  Training on CPU ...')
else:
    print('CUDA is available!  Training on GPU ...')
device = torch.device("cuda" if use_cuda else "cpu")

CUDA is available!  Training on GPU ...


In [3]:
# import zipfile
# with zipfile.ZipFile('./train_sample.zip', 'r') as zip_ref:
#     zip_ref.extractall('sample')

In [4]:
sample_data = np.array(glob("./train_sample/*"))
print('There are %d total images.' % len(sample_data))

There are 1913 total images.


In [5]:
# df = pd.read_csv('questiondf.csv')
df = pd.read_csv('./Files/questiondf.csv',index_col=0)

In [6]:
excl_ids = np.load('./Files/excluded_img_ids.npy')
drop_indices = []
for i,row in df.iterrows():
    if row['image_id'] in excl_ids:
        drop_indices.append(i)

df = df.drop(drop_indices)
# df = df.drop(columns=['question_type','answers'])

In [7]:
df.head()
# len(df)

,image_id,question,question_id
0,458752,What is this photo taken looking through?,458752000
1,458752,What position is this man playing?,458752001
2,458752,What color is the players shirt?,458752002
3,458752,Is this man a professional baseball player?,458752003
4,262146,What color is the snow?,262146000


In [8]:
# ids = list(df['image_id'])
# dfid = np.array(list(df['image_id']))
# ids = []
# for i in dfid:
#   if i not in ids:
#     ids.append(i)

ids = np.array(list(set(df['image_id'].values)))

In [9]:
ids.shape

(1913,)

In [ ]:
# from image ids
directory = './train_sample/'
images = []
for i in ids:
    name=str(i)
    l = len(name)
    filename=''.join(['0'for x in range(12-l)])+name+".jpg"
    images.append(Image.open(directory+filename))
    images[-1].load()

In [ ]:
# # from sample data
# images = []
# ids = []
# for f in sample_data:
#     ids.append(int(f.split("/")[-1].split(".")[0]))
#     images.append(Image.open(f))
#     images[-1].load()

In [ ]:
# np.save('test_image_IDs',ids)

In [ ]:
# np.equal(sorted(ids),sorted(np.load('/content/test_sample_image_ids.npy'))).all()

In [ ]:
# type(images[0]),len(ids),len(images)

In [ ]:
# !pip install img2vec_pytorch

In [ ]:
from img2vec_pytorch import Img2Vec
from PIL import Image

img2vec = Img2Vec(cuda=True) #Change this for GPU

In [ ]:
# v1 = img2vec.get_vec(images[:2])
# v2 = img2vec.get_vec(images[2:4])
# v1 = np.concatenate((v1,v2))
# v1.shape

In [ ]:
batch = 50
n = floor(len(images)/batch)
vectors = img2vec.get_vec(images[:batch])
k = batch
for i in range(n):
  v = img2vec.get_vec(images[k:k+batch])
  vectors = np.concatenate((vectors,v))
  k+=batch

In [ ]:
# len(vectors)
np.save('/content/train_img_vectors.npy',vectors)

In [ ]:
# vectors = np.load('/content/img_vectors_sample.npy')
vectors = np.load('train_img_vectors.npy')

In [ ]:
len(vectors)

In [ ]:
import pandas as pd
df = pd.DataFrame(zip(ids,vectors),columns=['Image ID','Image Vector'])

print(df.head())

df.to_csv('train_image_vectors_with_ids.csv',index=False)

In [ ]:
len(df)

In [ ]:
df = pd.DataFrame(zip(ids,vectors),columns=['Image ID','Image Vector'])
print(df.head())
df.to_csv('image_vectors_with_ids.csv',index=False)